In [1]:
from pathlib import Path
import re
import warnings

import matplotlib.pyplot as plt
import MeCab
import numpy as np
import pandas as pd
import seaborn as sns

from utils.jupyter.settings import PROJECT_ROOT, DATA_DIR, get_line

warnings.filterwarnings('ignore')

sns.set()
plt.rcParams['font.family'] = 'IPAPGothic'  # TO-DO: Check detail and decide to remove or not
%matplotlib inline

In [2]:
CELL_WIDTH = 124
DASHED_LINE = get_line(CELL_WIDTH)
DOUBLE_DASHED_LINE = get_line(CELL_WIDTH, line_type='=')

In [3]:
NEKO_FILEPATH = DATA_DIR.joinpath('neko.txt')
NEKO_MECAB_FILEPATH = DATA_DIR.joinpath('neko.txt.mecab')

# 第4章: 形態素解析

夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt)）を MeCab を使って形態素解析し, その結果を neko.txt.mecab というファイルに保存せよ. このファイルを用いて, 以下の問に対応するプログラムを実装せよ.

なお，問題[37](#37), [38](#38), [39](#39)は [matplotlib](http://matplotlib.org/) もしくは [Gnuplot](http://www.gnuplot.info/) を用いるとよい.

In [4]:
with NEKO_FILEPATH.open() as f:
    lines = f.read()[1:]  # Ignore the first line '-'    
lines = re.sub(r"[\n\u3000]", r"", lines)    

In [5]:
m = MeCab.Tagger()

In [6]:
tagged = m.parse(lines)

In [7]:
with NEKO_MECAB_FILEPATH.open('w') as f:
    f.write(tagged)

## 30. 形態素解析結果の読み込み

形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ. ただし, 各形態素は表層形（```surface```）, 基本形（```base```）, 品詞（```pos```）, 品詞細分類1（```pos1```）をキーとするマッピング型に格納し, 1文を形態素（マッピング型）のリストとして表現せよ. 第4章の残りの問題では, ここで作ったプログラムを活用せよ.

In [8]:
def get_mapping(line: str) -> dict:
    """Define a map or dict containing part-of-speech information."""
    line = line.strip()
    tab_splitted = line.split('\t')
    surface = tab_splitted[0]
    comma_splitted = tab_splitted[1].split(',')
    base, pos, pos1 = comma_splitted[-3], comma_splitted[0], comma_splitted[1]
    return {'surface': surface, 'base': base, 'pos': pos, 'pos1': pos1}

In [9]:
with NEKO_MECAB_FILEPATH.open() as f:
    tagged = f.readlines()

In [10]:
sents = []
sent = []
for line in tagged[:-1]:  # Ignore EOF symbol
    sent += [get_mapping(line)]
    if line[0] == '。':
        sents += [sent]
        sent = []

## 31. 動詞

動詞の表層形をすべて抽出せよ.

In [11]:
verb_surfaces = [
    mapping['surface'] for sent in sents 
                       for mapping in sent if mapping['pos'] == '動詞'
]

# Output only the first 100 results
print(verb_surfaces[:100])

['生れ', 'つか', 'し', '泣い', 'し', 'いる', '始め', '見', '聞く', '捕え', '煮', '食う', '思わ', '載せ', 'られ', '持ち上げ', 'られ', 'し', 'あっ', '落ちつい', '見', '見', '思っ', '残っ', 'いる', 'さ', 'れ', 'し', '逢っ', '出会わ', 'し', 'なら', 'し', 'いる', '吹く', 'せ', '弱っ', '飲む', '知っ', '坐っ', 'おっ', 'する', 'し', '始め', '動く', '動く', '分ら', '廻る', 'なる', '助から', '思っ', 'いる', 'さり', 'し', '出', 'し', 'いる', '考え出そ', '分ら', '付い', '見る', 'い', 'おっ', '見え', '隠し', 'しまっ', '違っ', '明い', 'い', 'られ', '這い出し', '見る', '棄て', 'られ', '這い出す', 'ある', '坐っ', 'し', '考え', '見', '出', 'し', '泣い', '来', 'くれる', '考え付い', 'やっ', '見', '来', '渡っ', 'かかる', '減っ', '来', '泣き', '出', 'ある', 'ある', 'し', 'そろ', '廻り']


## 32. 動詞の原形

動詞の原形をすべて抽出せよ.

In [12]:
verb_bases = [
    mapping['base'] for sent in sents
                    for mapping in sent if mapping['pos'] == '動詞'
]

# Output only the first 100 results
print(verb_bases[:100])

['生れる', 'つく', 'する', '泣く', 'する', 'いる', '始める', '見る', '聞く', '捕える', '煮る', '食う', '思う', '載せる', 'られる', '持ち上げる', 'られる', 'する', 'ある', '落ちつく', '見る', '見る', '思う', '残る', 'いる', 'する', 'れる', 'する', '逢う', '出会う', 'する', 'なる', 'する', 'いる', '吹く', 'する', '弱る', '飲む', '知る', '坐る', 'おる', 'する', 'する', '始める', '動く', '動く', '分る', '廻る', 'なる', '助かる', '思う', 'いる', 'さる', 'する', '出る', 'する', 'いる', '考え出す', '分る', '付く', '見る', 'いる', 'おる', '見える', '隠す', 'しまう', '違う', '明く', 'いる', 'られる', '這い出す', '見る', '棄てる', 'られる', '這い出す', 'ある', '坐る', 'する', '考える', '見る', '出る', 'する', '泣く', '来る', 'くれる', '考え付く', 'やる', '見る', '来る', '渡る', 'かかる', '減る', '来る', '泣く', '出る', 'ある', 'ある', 'する', 'そる', '廻る']


## 33. サ変名詞

サ変接続の名詞をすべて抽出せよ.

In [13]:
nouns_sahen_concated = [
    mapping['base'] for sent in sents for mapping in sent 
                    if mapping['pos'] == '名詞' and mapping['pos1'] == 'サ変接続'
]

# Output only the first 100 results
print(nouns_sahen_concated[:100])

['見当', '記憶', '話', '装飾', '突起', '運転', '記憶', '分別', '決心', '我慢', '餓死', '訪問', '始末', '猶予', '遭遇', '我慢', '記憶', '返報', '勉強', '勉強', '昼寝', '珍重', '昼寝', '経験', '供', '供', '供', '供', '*', '*', '同居', '観察', '断言', '同衾', '供', '迫害', '尊敬', '生活', '剿滅', '議論', '所有', '憤慨', '観念', '御馳走', '掠奪', '代言', '我儘', '我儘', '失敗', '話', '投書', '関', '決心', '昼寝', '鑑定', '話', '述懐', '想像', '写生', '感心', '昼寝', '失笑', '揶揄', '写生', '欠伸', '自白', '彩色', '写生', '判然', '心中', '感服', '小便', '猶予', '失敬', '欠伸', '予定', '失望', '漫罵', '小便', '増長', '増長', '我儘', '我慢', '報道', '供', '昼寝', '退屈', '加減', '一睡', '運動', '嘆賞', '佇立', '記憶', '珍重', '身動き', '挨拶', '鼓動', '軽蔑', '察', '肥満']


## 34. 「AのB」

2つの名詞が「の」で連結されている名詞句を抽出せよ.

## 35. 名詞の連接

名詞の連接（連続して出現する名詞）を最長一致で抽出せよ.

## 36. 単語の出現頻度

文章中に出現する単語とその出現頻度を求め, 出現頻度の高い順に並べよ.

## 37. 頻度上位10語

出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ.

## 38. ヒストグラム

単語の出現頻度のヒストグラム（横軸に出現頻度, 縦軸に出現頻度をとる単語の種類数を棒グラフで表したもの）を描け.

## 39. Zipf の法則

単語の出現頻度順位を横軸, その出現頻度を縦軸として, 両対数グラフをプロットせよ.